<a href="https://colab.research.google.com/github/fikarrizkip/projects/blob/main/Multiclass_Classification_dengan_Algoritma_Multinomial_Naive_Bayes_dan_k_Nearest_Neighbors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_excel('https://storage.googleapis.com/dqlab-dataset/credit_scoring_dqlab.xlsx')

In [ ]:
print(df.head(5))

  kode_kontrak  pendapatan_setahun_juta kpr_aktif  durasi_pinjaman_bulan  \
0   AGR-000001                      295        YA                     48   
1   AGR-000011                      271        YA                     36   
2   AGR-000030                      159     TIDAK                     12   
3   AGR-000043                      210        YA                     12   
4   AGR-000049                      165     TIDAK                     36   

   jumlah_tanggungan rata_rata_overdue  risk_rating  
0                  5      61 - 90 days            4  
1                  5      61 - 90 days            4  
2                  0       0 - 30 days            1  
3                  3      46 - 60 days            3  
4                  0      31 - 45 days            2  


In [ ]:
df.drop('kode_kontrak', axis=1, inplace=True)

In [ ]:
#memeriksa rasio kemunculan label
print('Rasio kemunculan  label:')
print(pd.concat([df['risk_rating'].value_counts(), 100*df['risk_rating'].value_counts(normalize=True).rename('percentage_risk_rating')], axis=1))

#menyimpan kolom 'rating' sebagai list ke dalam variabel y
y = df.pop('risk_rating').to_list()

#untuk setiap label jika nilai label = 5 maka kembalikan nilai 4
y = [4 if label == 5 else label for label in y]

#mengubah tipe data dari array y menjadi numpy array hal ini dikarenakan beberapa fungsi library scikit-learn hanya kompatibel terhadap numpy array
import numpy as np
y = np.array(y)

print('\nDataset:')
print(df.head())

Rasio kemunculan  label:
   risk_rating  percentage_risk_rating
3          291               32.333333
1          227               25.222222
2          159               17.666667
4          120               13.333333
5          103               11.444444

Dataset:
   pendapatan_setahun_juta kpr_aktif  durasi_pinjaman_bulan  \
0                      295        YA                     48   
1                      271        YA                     36   
2                      159     TIDAK                     12   
3                      210        YA                     12   
4                      165     TIDAK                     36   

   jumlah_tanggungan rata_rata_overdue  
0                  5      61 - 90 days  
1                  5      61 - 90 days  
2                  0       0 - 30 days  
3                  3      46 - 60 days  
4                  0      31 - 45 days  


In [ ]:
#menyiapkan fungsi untuk mengubah nilai kpr_aktif
def convert_kpr_aktif(kpr_aktif):
	if kpr_aktif == 'YA':
		return 1
	return 0

#mengubah setiap nilai pada kolom kpr aktif menggunakan fungsi 'convert_kpr_aktif'
df['kpr_aktif']= df['kpr_aktif'].apply(convert_kpr_aktif)
print(df.head())

   pendapatan_setahun_juta  kpr_aktif  durasi_pinjaman_bulan  \
0                      295          1                     48   
1                      271          1                     36   
2                      159          0                     12   
3                      210          1                     12   
4                      165          0                     36   

   jumlah_tanggungan rata_rata_overdue  
0                  5      61 - 90 days  
1                  5      61 - 90 days  
2                  0       0 - 30 days  
3                  3      46 - 60 days  
4                  0      31 - 45 days  


In [ ]:
#rasio kemunculan setiap kategori rata_rata_overdue
print('Rasio kemunculan  setiap kategori rata_rata_overdue:')
print(pd.concat([df['rata_rata_overdue'].value_counts(), 100*df['rata_rata_overdue'].value_counts(normalize=True).rename('percentage_rata_rata_overdue')], axis=1))

#fungsi untuk mengubah nilai dari kolom 'rata_rata_overdue'
def change_overdue(overdue):
	if overdue == '0 - 30 days':
		return 0
	elif overdue == '31 - 45 days':
		return 1
	elif overdue == '46 - 60 days':
		return 2
	elif overdue == '61 - 90 days':
		return 3
	else:
		return 4

#mengaplikasikan fungsi pada kolom 'rata_rata_overdue'
df['rata_rata_overdue'] = df['rata_rata_overdue'].apply(change_overdue)

print("\nLima baris dataset teratas:")
print(df.head())

Rasio kemunculan  setiap kategori rata_rata_overdue:
              rata_rata_overdue  percentage_rata_rata_overdue
46 - 60 days                291                     32.333333
0 - 30 days                 227                     25.222222
31 - 45 days                159                     17.666667
61 - 90 days                120                     13.333333
> 90 days                   103                     11.444444

Lima baris dataset teratas:
   pendapatan_setahun_juta  kpr_aktif  durasi_pinjaman_bulan  \
0                      295          1                     48   
1                      271          1                     36   
2                      159          0                     12   
3                      210          1                     12   
4                      165          0                     36   

   jumlah_tanggungan  rata_rata_overdue  
0                  5                  3  
1                  5                  3  
2                  0               

In [ ]:
#merepresentasikan seluruh kolom dalam df sebagai matrik dan menampungnya dalam variabel X.
X = df.to_numpy()

#menampilkan informasi dimensi dari variabel X
print('Dimensi dari variabel X:', X.shape)

Dimensi dari variabel X: (900, 5)


In [ ]:
#library yang akan kita gunakan untuk membagi dataset menggunakan strategi validasi KFold
from sklearn.model_selection import KFold

#menginisialisasi object KFold dengan jumlah kelompok data = 5 nilai random_state kita gunakan reproducibility (agar data acak yang kita dapatkan untuk setiap kelompok data selalu sama)
kf = KFold(n_splits = 5, shuffle=True, random_state = 57)

#meminta object kf untuk memecah data X ke sejumlah n kelompok dan mengiterasi setiap train_index dan test_index
for i, (train_index, test_index) in enumerate(kf.split(X)):
	X_train, y_train = X[train_index], y[train_index]
	X_test, y_test = X[test_index], y[test_index]

	print("Percobaan ke-", i+1)
	print("10 indeks data latih pertama:", train_index[:10])
	print("10 indeks data testing pertama:", test_index[:10])
	print("============================================================")

Percobaan ke- 1
10 indeks data latih pertama: [ 0  1  2  3  4  5  6  7  8 13]
10 indeks data testing pertama: [ 9 10 11 12 15 25 29 32 44 52]
Percobaan ke- 2
10 indeks data latih pertama: [ 0  2  4  5  6  8  9 10 11 12]
10 indeks data testing pertama: [ 1  3  7 14 20 28 37 43 49 59]
Percobaan ke- 3
10 indeks data latih pertama: [ 0  1  2  3  4  5  6  7  9 10]
10 indeks data testing pertama: [ 8 13 22 23 27 30 31 33 38 39]
Percobaan ke- 4
10 indeks data latih pertama: [ 1  3  4  5  6  7  8  9 10 11]
10 indeks data testing pertama: [ 0  2 16 17 19 21 24 35 36 42]
Percobaan ke- 5
10 indeks data latih pertama: [ 0  1  2  3  7  8  9 10 11 12]
10 indeks data testing pertama: [ 4  5  6 18 26 34 40 47 50 54]


In [ ]:
#library yang digunakan untuk menggunakan model MNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

scores_test = []
scores_train = []

#meminta object kf untuk memecah data X ke sejumlah n kelompok dan mengiterasi setiap train_index dan test_index
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    model = MultinomialNB()
    #menspesifikasikan data latih beserta labelnya untuk dipelajari oleh model
    model.fit(X_train, y_train)
    print(f'Selesai melatih data dengan strategi validasi 5-Fold ke-{i+1}.')

Selesai melatih data dengan strategi validasi 5-Fold ke-1.
Selesai melatih data dengan strategi validasi 5-Fold ke-2.
Selesai melatih data dengan strategi validasi 5-Fold ke-3.
Selesai melatih data dengan strategi validasi 5-Fold ke-4.
Selesai melatih data dengan strategi validasi 5-Fold ke-5.


In [ ]:
#meminta object kf untuk memecah data X ke sejumlah n kelompok dan mengiterasi setiap train_index dan test_index
for train_index, test_index in kf.split(X):
	X_train, y_train = X[train_index], y[train_index]
	X_test, y_test = X[test_index], y[test_index]
	model = MultinomialNB()

	#menspesifikasikan data latih beserta labelnya untuk dipelajari oleh model
	model.fit(X_train, y_train)

	#meminta model yang telah dilatih untuk memprediksi data X_train
	#menampung hasil prediksi ke dalam variabel y_pred
	y_pred = model.predict(X_train)

	#menampung akurasi dari model ke variabel current_score
	current_score = accuracy_score(y_train, y_pred)

	#menambahkan skor saat ini ke list scores
	scores_train.append(current_score)

	#meminta model yang telah dilatih untuk memprediksi data X_test
	#menampung hasil prediksi ke dalam variabel y_pred
	y_pred = model.predict(X_test)

	#menampung akurasi dari model ke variabel current_score
	current_score = accuracy_score(y_test, y_pred)

	#menambahkan skor saat ini ke list scores
	scores_test.append(current_score)

print("Skor latih tertinggi: ", round(max(scores_train),2))
print("Skor latih terendah: ", round(min(scores_train),2))
print("Skor latih rata-rata: ", round(sum(scores_train)/len(scores_train),2))

print("Skor testing tertinggi: ", round(max(scores_test),2))
print("Skor testing terendah: ", round(min(scores_test),2))
print("Skor testing rata-rata: ", round(sum(scores_test))/len(scores_test),2)

Skor latih tertinggi:  0.71
Skor latih terendah:  0.68
Skor latih rata-rata:  0.7
Skor testing tertinggi:  0.76
Skor testing terendah:  0.65
Skor testing rata-rata:  0.6 2


Inisialisasi, Pelatihan dan Evaluasi Model Klasifikasi dengan Metode k-NN

In [ ]:
#library yang digunakan untuk menggunakan model k-NN
from sklearn.neighbors import KNeighborsClassifier

#library yang akan kita gunakan untuk membagi dataset menggunakan strategi validasi KFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

#menginisialisasi object KFold dengan jumlah kelompok data = 5. nilai random_state kita gunakan reproducibility (agar data acak yang kita dapatkan untuk setiap kelompok data selalu sama)
kf = KFold(n_splits = 5, shuffle=True, random_state = 57)

scores_test = []
scores_train = []

#meminta object kf untuk memecah data X ke sejumlah n kelompok dan mengiterasi setiap train_index dan test_index
for train_index, test_index in kf.split(X):
	X_train, y_train = X[train_index], y[train_index]
	X_test, y_test = X[test_index], y[test_index]

	model = KNeighborsClassifier()

	#menspesifikasikan data latih beserta labelnya untuk dipelajari oleh model
	model.fit(X_train, y_train)

	#meminta model yang telah dilatih untuk memprediksi data X_train
	#menampung hasil prediksi ke dalam variabel y_pred
	y_pred = model.predict(X_train)

	#menampung akurasi dari model ke variabel current_score
	current_score = accuracy_score(y_train, y_pred)

	#menambahkan skor saat ini ke list scores
	scores_train.append(current_score)

	#meminta model yang telah dilatih untuk memprediksi data X_test
	#menampung hasil prediksi ke dalam variabel y_pred
	y_pred = model.predict(X_test)

	#menampung akurasi dari model ke variabel current_score
	current_score = accuracy_score(y_test, y_pred)

	#menambahkan skor saat ini ke list scores
	scores_test.append(current_score)

print("Skor latih tertinggi: ", round(max(scores_train),2))
print("Skor latih terendah: ", round(min(scores_train),2))
print("Skor latih rata-rata: ", round(sum(scores_train)/ len(scores_train),2))

print("Skor testing tertinggi: ", round(max(scores_test),2))
print("Skor testing terendah: ", round(min(scores_test),2))
print("Skor testing rata-rata: ", round(sum(scores_test)/ len(scores_test),2))

Skor latih tertinggi:  0.75
Skor latih terendah:  0.72
Skor latih rata-rata:  0.74
Skor testing tertinggi:  0.62
Skor testing terendah:  0.57
Skor testing rata-rata:  0.59


In [ ]:
clf = KNeighborsClassifier()
#parameter-parameter yang akan diujicobakan pada model
parameter_space = {
    'n_neighbors': [5, 10, 15, 20, 25],
    'metric': ['euclidean', 'manhattan']
}

#menginisialisasi object GridSearchCV pada classifier
#penjelasan terkait dengan parameter-parameter lain yang diterima pada object akan diberikan setelah potongan kode

kfold = KFold(n_splits=5, shuffle=True,random_state=57)
searcher = GridSearchCV(clf, parameter_space, scoring='accuracy', cv = kfold)

#meminta object untuk memproses data X dan y.
searcher.fit(X,y)

print("Parameter terbaik: ", searcher.best_params_)
print("Akurasi terbaik: ", searcher.best_score_)

Parameter terbaik:  {'metric': 'manhattan', 'n_neighbors': 5}
Akurasi terbaik:  0.7333333333333333


In [ ]:
from sklearn.pipeline import Pipeline

#library yang kita gunakan untuk mengubah skalasi nilai dari setiap atribut ke dalam suatu rentang tertentu
from sklearn.preprocessing import MinMaxScaler

clf = Pipeline([
#menginisialisasi object MinMaxScaler untuk mengubah setiap kolom agar nilai maksimumnya sama dengan 1 dan nilai minimumnya sama dengan 0
  ('scaler', MinMaxScaler(feature_range=(0,1))),
  ('clf', KNeighborsClassifier()),
])

parameter_space = {
    'clf__n_neighbors': [2,3,4,5,6,7,8,9,11],
    'clf__metric': ['euclidean','manhattan']
}

kfold = KFold(n_splits=5, shuffle=True,random_state=57)
searcher = GridSearchCV(clf, parameter_space, scoring='accuracy', cv = kfold)

searcher.fit(X,y)

print("Parameter terbaik: ", searcher.best_params_)
print("Akurasi terbaik: ", searcher.best_score_)

Parameter terbaik:  {'clf__metric': 'manhattan', 'clf__n_neighbors': 6}
Akurasi terbaik:  0.9844444444444445


**Presentasi Final Model Klasifikasi Credit Scoring**

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

clf = Pipeline([
    ('scaler', MinMaxScaler(feature_range=(0,1))),
    ('clf', KNeighborsClassifier()),
])

parameter_space = {
    'clf__n_neighbors': [2,3,4,5,6,7,8,9,11],
    'clf__metric': ['euclidean', 'manhattan']
}

kfold = KFold(n_splits=5, shuffle=True,random_state=57)
searcher = GridSearchCV(clf, parameter_space, scoring='accuracy', cv = kfold)
searcher.fit(X,y)

#merepresentasikan ketiga data sebagai variabel X_new
X_new = [
    [320,1,36,1,2],
    [220,0,12,0,1],
    [300,0,12,2,0]
]

#perhatikan bahwa kita tidak harus melakukan skalasi terhadap X_new dan proses predict akan melakukannya untuk kita karena kita sudah menspesifikasikan object MinMaxScaler dalam object pipeline pada potongan kode sebelumnya
print(searcher.best_estimator_.predict(X_new))

[3 2 1]


**Mini Project: Multinomial Naive Bayes**

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_column', 20)

df = pd.read_excel('https://storage.googleapis.com/dqlab-dataset/credit_scoring_dqlab.xlsx')
df.drop('kode_kontrak', axis=1, inplace=True)

y = df.pop('risk_rating').to_list()
y = [4 if label == 5 else label for label in y]
y = np.array(y)

def convert_kpr_aktif(kpr_aktif):
    if kpr_aktif == 'YA':
        return 1
    return 0

df['kpr_aktif']= df['kpr_aktif'].apply(convert_kpr_aktif)

def change_overdue(overdue):
    if overdue == '0 - 30 days':
        return 0
    elif overdue == '31 - 45 days':
        return 1
    elif overdue == '46 - 60 days':
        return 2
    elif overdue == '61 - 90 days':
        return 3
    else:
        return 4

df['rata_rata_overdue'] = df['rata_rata_overdue'].apply(change_overdue)

X = df.to_numpy()

#library yang digunakan untuk menghitung akurasi
from sklearn.metrics import accuracy_score

#library yang digunakan untuk menggunakan model MNB
from sklearn.naive_bayes import MultinomialNB

#library yang akan kita gunakan untuk membagi dataset menggunakan strategi validasi KFold
from sklearn.model_selection import KFold

#menginisialisasi object KFold dengan jumlah kelompok data = 10. nilai random_state kita gunakan reproducibility (agar data acak yang kita dapatkan untuk setiap kelompok data selalu sama)
kf = KFold(n_splits = 10, shuffle=True, random_state = 57)

scores_test = []
scores_train = []

#meminta object kf untuk memecah data X ke sejumlah n kelompok dan mengiterasi setiap train_index dan test_index
for train_index, test_index in kf.split(X):
	X_train, y_train = X[train_index], y[train_index]
	X_test, y_test = X[test_index], y[test_index]
	model = MultinomialNB()

	#menspesifikasikan data latih beserta labelnya untuk dipelajari oleh model
	model.fit(X_train, y_train)

	#meminta model yang telah dilatih untuk memprediksi data X_train
	#menampung hasil prediksi ke dalam variabel y_pred
	y_pred = model.predict(X_train)

	#menampung akurasi dari model ke variabel current_score
	current_score = accuracy_score(y_train, y_pred)

	#menambahkan skor saat ini ke list scores
	scores_train.append(current_score)

	#meminta model yang telah dilatih untuk memprediksi data X_test
	#menampung hasil prediksi ke dalam variabel y_pred
	y_pred = model.predict(X_test)

	#menampung akurasi dari model ke variabel current_score
	current_score = accuracy_score(y_test, y_pred)

	#menambahkan skor saat ini ke list scores
	scores_test.append(current_score)

print("Skor latih tertinggi: ", round(max(scores_train),2))
print("Skor latih terendah: ", round(min(scores_train),2))
print("Skor latih rata-rata: ", round(sum(scores_train)/len(scores_train),2))

print("Skor testing tertinggi: ", round(max(scores_test),2))
print("Skor testing terendah: ", round(min(scores_test),2))
print("Skor testing rata-rata: ", round(sum(scores_test))/len(scores_test),2)

Skor latih tertinggi:  0.7
Skor latih terendah:  0.69
Skor latih rata-rata:  0.69
Skor testing tertinggi:  0.79
Skor testing terendah:  0.61
Skor testing rata-rata:  0.7 2


Mini Project: k-Nearest **Neighbors**

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_column', 20)

df = pd.read_excel('https://storage.googleapis.com/dqlab-dataset/credit_scoring_dqlab.xlsx')
df.drop('kode_kontrak', axis=1, inplace=True)

y = df.pop('risk_rating').to_list()
y = [4 if label == 5 else label for label in y]
y = np.array(y)

def convert_kpr_aktif(kpr_aktif):
    if kpr_aktif == 'YA':
        return 1
    return 0

df['kpr_aktif']= df['kpr_aktif'].apply(convert_kpr_aktif)

def change_overdue(overdue):
    if overdue == '0 - 30 days':
        return 0
    elif overdue == '31 - 45 days':
        return 1
    elif overdue == '46 - 60 days':
        return 2
    elif overdue == '61 - 90 days':
        return 3
    else:
        return 4

df['rata_rata_overdue'] = df['rata_rata_overdue'].apply(change_overdue)

X = df.to_numpy()

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

clf = KNeighborsClassifier()

#parameter-parameter yang akan diujicobakan pada model
parameter_space = {
    'n_neighbors': [2,3,4,5,6,7,8,9,11],
    'metric': ['euclidean', 'manhattan']
}

#menginisialisasi object GridSearchCV pada classifier, penjelasan terkait dengan parameter-parameter lain yang diterima pada object akan diberikan setelah potongan kode
kfold = KFold(n_splits=5, shuffle=True,random_state=57)
searcher = GridSearchCV(clf, parameter_space, scoring='accuracy', cv = kfold)

#meminta object untuk memproses data X dan y.
searcher.fit(X,y)

print("Parameter terbaik: ", searcher.best_params_)
print("Akurasi terbaik: ", searcher.best_score_)

Parameter terbaik:  {'metric': 'manhattan', 'n_neighbors': 3}
Akurasi terbaik:  0.7755555555555554
